In [1]:
class Blocker:
    def __init__(self):
        self.blocked_counter = {
            'up': 0,
            'down': 0,
            'left': 0,
            'right': 0,
        }

In [2]:
with open("../../data/day06-input.txt") as f:
    GRID = []
    i = 0
    for line in f.readlines():
        line = line.strip()
        row = []
        j = 0
        for char in line:
            if char == '.':
                row.append(False)
            elif char == '^':
                STARTING_POSITION = (i, j)
                STARTING_HEADING = "up"
                row.append(False)
            else:
                # There is an obstruction at this spot
                row.append(True)
            j += 1
        i += 1
        GRID.append(row)
    del line, i, j

### Note
After solving this the brute force way, I realize optimizing it could be done by:
(1) instead of simulating the path walk, I could have used the row and column values
to determine if the entire path is clear or not, and (2) only considering the squares
where the agent moved to be candidates for placing a new blocker to cause a loop.
Since the vectors of travel can be recorded on the blocker objects for the initial
run through, we can use that information to locate three blockers that meet criteria
for the construction of the loop. In addition to being positioned, that criteria
requires the blockers to have mutually exclusive movement counters so that upon
placing the fourth, the agent will trigger all four cardinal direction counters just once.

In [3]:
DIMENSIONS = (len(GRID), len(GRID[0]))
MOVEMENT = {
    'up': lambda i, j: (i - 1, j),
    'down': lambda i, j: (i + 1, j),
    'left': lambda i, j: (i, j - 1),
    'right': lambda i, j: (i, j + 1),
}
MOVEMENT_ORDER = ['up', 'right', 'down', 'left']
MARKER = 'X'

In [4]:
def get_next_heading(heading):
    return MOVEMENT_ORDER[(MOVEMENT_ORDER.index(heading) + 1) % len(MOVEMENT_ORDER)]

def get_next_position(position: tuple[int, int], heading):
    return MOVEMENT[heading](*position)

def is_pos_out_of_bounds(position: tuple[int, int]):
    if position[0] < 0 or position[0] >= DIMENSIONS[0] \
            or position[1] < 0 or position[1] >= DIMENSIONS[1]:
        return True
    return False

def is_pos_occupied(position: tuple[int, int], grid: list[list]) -> bool:
    value =grid[position[0]][position[1]]
    return isinstance(value, Blocker)

def init_grid() -> list[list[Blocker | bool]]:
    new_grid = []
    for row in GRID:
        new_row = []
        for col in row:
            if col == True:
                new_row.append(Blocker())
            else:
                new_row.append(False)
        new_grid.append(new_row)
    return new_grid

def count_marked_positions(grid: list[list[bool or 'X']]) -> int:
    count = 0
    for row in grid:
        for col in row:
            if col == 'X':
                count += 1
    return count

def mark_position(position: tuple[int, int], grid: list[list[bool or 'X']]):
    grid[position[0]][position[1]] = MARKER

In [5]:
i = 0
max_iter = 1e8
current_heading = STARTING_HEADING
current_position = STARTING_POSITION
current_grid = init_grid()
mark_position(current_position, current_grid)
while i < max_iter:
    next_position = get_next_position(current_position, current_heading)
    if is_pos_out_of_bounds(next_position):
        mark_position(current_position, current_grid)
        break
    if is_pos_occupied(next_position, current_grid):
        current_heading = get_next_heading(current_heading)
        continue

    mark_position(current_position, current_grid)
    current_position = next_position
    i += 1


print('Took %s iterations' % i)
print('Marked %s positions' % count_marked_positions(current_grid))

Took 5456 iterations
Marked 4939 positions


In [6]:
with open('./day6_current_grid.txt', 'w') as f:
    for row in current_grid:
        for col in row:
            if col == 'X':
                f.write('X')
            elif isinstance(col, Blocker):
                f.write('#')
            else:
                f.write('.')
        f.write('\n')

    f.write('\n')

In [7]:
max_iter = 1e8
counter = 0
i = 0

def iter_grid_positions():
    for x in range(DIMENSIONS[0]):
        for y in range(DIMENSIONS[1]):
            yield (x, y)

for trial_position in iter_grid_positions():
    # It is a blocked space
    if GRID[trial_position[0]][trial_position[1]]:
        continue
    # It is the starting position
    if trial_position == STARTING_POSITION:
        continue

    causes_loop = False
    current_heading = STARTING_HEADING
    current_position = STARTING_POSITION
    current_grid = init_grid()
    current_grid[trial_position[0]][trial_position[1]] = Blocker()
    mark_position(current_position, current_grid)
    while i < max_iter:
        next_position = get_next_position(current_position, current_heading)
        if is_pos_out_of_bounds(next_position):
            mark_position(current_position, current_grid)
            break
        if is_pos_occupied(next_position, current_grid):
            item = current_grid[next_position[0]][next_position[1]]
            if item.blocked_counter[current_heading] > 0:
                # loop detected
                causes_loop = True
                break
            item.blocked_counter[current_heading] += 1
            current_heading = get_next_heading(current_heading)
            continue

        mark_position(current_position, current_grid)
        current_position = next_position
        i += 1

    if causes_loop:
        counter += 1

assert i != max_iter, 'Reached max iterations'

print('Took %s iterations' % i)
print('Total positions causing a loop: %s' % counter)

Took 77071576 iterations
Total positions causing a loop: 1434
